# Setup

## Imports

In [ ]:
import sys
import icecream
sys.path.append('./translator')

In [ ]:
import torch
import rp
from translator.trainer import MUNIT_Trainer as Trainer
from translator.data import ImageFolder

In [ ]:
from IPython.display import clear_output

## Other Setup

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format='retina'

# Functions

In [ ]:
config_file       = './translator/configs/alphablock_without_ssim_256.yaml'
image_folder_path = './datasets/alphacube/scenes/'
# image_folder_path = './datasets/alphacube/anim_2/'
# image_folder_path = './datasets/alphacube/anim_1/'
image_folder_path = '/mnt/Noman/Ubuntu/CleanCode/Datasets/diff_rendering/alphabetcube_L/SyntheticData/Anim3/Renderings'
checkpoint_folder = './translator/trained_models/outputs/alphablock_without_ssim_256/checkpoints'
# checkpoint_folder = './translator/trained_models/outputs/alphablock_without_ssim_256/checkpoints/old_checkpoints/v0.0.7'
# checkpoint_folder = './translator/save'

In [ ]:
config = rp.load_yaml_file(config_file)

In [ ]:
trainer=Trainer(config, trainable=False).to(device)

In [ ]:
trainer.resume(checkpoint_folder, config)

In [ ]:
rp.display_image(rp.tiled_images(rp.as_numpy_images(trainer.texture_pack())))

## TODO: Make the Trainer save the neural texture once, so during inference we don't need to run it over and over again (it's redundant to do that seeing as the texture never changes)

In [ ]:
aug = {}
aug["new_size_min"] = config["new_size_min_a"]
aug["new_size_max"] = config["new_size_max_a"]
aug["output_size" ] = (-1,-1) #Meaningless when skip_crop = True
# aug["output_size" ] = (320,320) #Meaningless when skip_crop = True
image_folder = ImageFolder(root=image_folder_path, precise=True, augmentation=aug)
image_folder.skip_crop = True

## Test 1

In [ ]:
o=rp.random_element(image_folder)

In [ ]:
_, height, width = o.shape
icecream.ic(height,width)

In [ ]:
rp.display_image(rp.as_numpy_image(o))

In [ ]:
h=o[None].to(device)
i=trainer.sample_a2b(h)


rp.display_image(rp.as_numpy_image(h[0]))
rp.display_image(rp.as_numpy_image(i[0]))

In [ ]:
i=trainer.sample(h,h)[:7]
for x in i:
    rp.display_image(rp.as_numpy_image(x[0]))

## Test 2

In [ ]:
def translate(image):
    #Input image is a UV-L Scene
    assert rp.is_image(image)
    image = rp.as_rgb_image  (image)
    image = rp.as_float_image(image)
    
    image = rp.as_torch_image(image)[None] #BCHW
    image = image.to(device)
    
    output = trainer.sample_a2b(image)
    output = output[0]
    output = rp.as_numpy_image(output)
    
    return output

In [ ]:
images = rp.load_images(image_folder_path, show_progress=True)
clear_output()

images = [rp.cv_resize_image(image, (height, width), 'nearest') for image in images]

In [ ]:
display_eta = rp.eta(len(images))
translations = []
for index,image in enumerate(images):
    display_eta(index)
    translations.append(translate(image))
clear_output()

In [ ]:
#Please be patient, this step can take a long time to complete (up to 5 minutes)
#This is because it's extremely inefficient lol. All images in this animation are being converted into base-64 in html
# rp.display_image_slideshow(translations[::5])

In [ ]:
video_path = 'untracked/animation.mp4'
rp.save_video(translations,'untracked/animation.mp4')
clear_output()
# translations=[rp.as_byte_image(x) for x in translations]
# translations=rp.as_numpy_array(translations)
# write_mp4(video_path, translations)

from IPython.display import Video
icecream.ic(video_path)
Video(video_path)